In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Use langchain RAG¶

In [2]:
pip install -q langchain langchain-community langchain_huggingface chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.9 MB/s eta 0:00:00

In [3]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

# 1. 한국어 문장 임베딩 모델 (항상 포함)
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# 2. 한국어 T5 모델 로드 (paust/pko-t5-base, encoder-decoder 구조, CPU 사용)
model_id = "paust/pko-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id, truncation=True, max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cpu")

# 3. LangChain용 텍스트 생성 파이프라인 구성 (CPU 사용, text2text-generation)
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1,                # -1이면 CPU 사용
    max_new_tokens=256,
    do_sample=True,           # 샘플링 활성화
    temperature=0.7,
    top_p=0.9
)

# 4. LangChain 호환 LLM 객체 생성 (return_full_text 옵션 포함)
hf_llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"return_full_text": False}
)

2025-04-13 12:06:07.063400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744545967.434572      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744545967.560759      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_13/814627585.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = Hu

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipykernel_13/814627585.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(


In [6]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 임베딩 모델
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

def load_Korean_wiki_docs(topic):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    content = soup.find("div", {"class": "mw-parser-output"})
    paragraphs = content.find_all("p")
    text = "\n".join([p.get_text() for p in paragraphs])

    documents = [Document(page_content=text, metadata={"source": url})]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    return splits

In [7]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [8]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=hf_embeddings,
        persist_directory="/tmp/chroma_db"  # 쓰기 가능한 경로 지정
    )
    vectorstore.persist()
    return vectorstore

In [9]:
topic = "폭싹 속았수다"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

/tmp/ipykernel_13/1328096435.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [10]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

# retriever 생성 (상위 4개 문서)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [11]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [12]:
question = "연출은 누가했습니까?"
result = qa_chain.invoke({"query": question})

print("결과:", result["result"])

결과: 넷플릭스 금요드라마는?

영어 제목인 ́폭싹 속았수다》는                   ,))은,


In [13]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

/tmp/ipykernel_13/2742417880.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/폭싹 속았수다'}, page_content='《폭싹 속았수다》(영어: When Life Gives You Tangerines)는 2025년 3월 7일부터 2025년 3월 28일까지 공개된 넷플릭스 금요드라마다. 이는 로맨스 일상물 TV 드라마로, 임상춘이 극본을 쓰고 김원석이 연출을 맡았으며, 아이유, 박보검, 문소리, 박해준이 출연한다. 이 드라마는 2025년 3월 7일부터 28일까지 넷플릭스를 통해 공개되었다.\n\n또, 국내외에서 큰 인기를 끌며 연기, 각본, 연출에 대한 찬사를 받았고, 한국적 정서에 뿌리를 둔 향수와 따뜻함을 자아낸다는 점에서 박보검이 출연했던 2015–2016년의 명작 시대극 《응답하라 1988》과 긍정적으로 비교되었다.\n\n제목인 《폭싹 속았수다》는 제주어로 \'무척 수고하셨습니다\'라는 뜻이다. "속았수다"의 어근인 \'속다\'는 제주어로 \'애쓰다\', \'수고하다\', \'욕보다\' 등의 뜻이 있다.[2] 한편 \'속다\'는 대한민국 표준어로 \'거짓을 참으로 알다\', \'남의 거짓이나 꾀에 넘어가다\'라는 뜻의 단어와 언어학적인 거짓짝 관계여서 《폭싹 속았수다》는 \'완전히 속아 넘어갔다\'로 잘못 해석될 여지가 있다.[2]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/폭싹 속았수다'}, page_content='영어 제목인 《When Life Gives You Tangerines》는 서구권의 속담인 "When life gives you lemons, make lemonade"(→삶이 레몬을 주거든 레모네이드를 만들라)에서 레몬(lemon)을 탠저린(tangerine)으로 치환한 것이다.[3] 그러나 제주도에서 재배되는 품종인 감귤은 영어로 \'mandarin (orange)\'(맨더린 [오렌지])이라고 일컫는다. 비슷한 예로 중국어 제목인 《苦

In [14]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/폭싹 속았수다'}, page_content='《폭싹 속았수다》(영어: When Life Gives You Tangerines)는 2025년 3월 7일부터 2025년 3월 28일까지 공개된 넷플릭스 금요드라마다. 이는 로맨스 일상물 TV 드라마로, 임상춘이 극본을 쓰고 김원석이 연출을 맡았으며, 아이유, 박보검, 문소리, 박해준이 출연한다. 이 드라마는 2025년 3월 7일부터 28일까지 넷플릭스를 통해 공개되었다.\n\n또, 국내외에서 큰 인기를 끌며 연기, 각본, 연출에 대한 찬사를 받았고, 한국적 정서에 뿌리를 둔 향수와 따뜻함을 자아낸다는 점에서 박보검이 출연했던 2015–2016년의 명작 시대극 《응답하라 1988》과 긍정적으로 비교되었다.\n\n제목인 《폭싹 속았수다》는 제주어로 \'무척 수고하셨습니다\'라는 뜻이다. "속았수다"의 어근인 \'속다\'는 제주어로 \'애쓰다\', \'수고하다\', \'욕보다\' 등의 뜻이 있다.[2] 한편 \'속다\'는 대한민국 표준어로 \'거짓을 참으로 알다\', \'남의 거짓이나 꾀에 넘어가다\'라는 뜻의 단어와 언어학적인 거짓짝 관계여서 《폭싹 속았수다》는 \'완전히 속아 넘어갔다\'로 잘못 해석될 여지가 있다.[2]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/폭싹 속았수다'}, page_content='영어 제목인 《When Life Gives You Tangerines》는 서구권의 속담인 "When life gives you lemons, make lemonade"(→삶이 레몬을 주거든 레모네이드를 만들라)에서 레몬(lemon)을 탠저린(tangerine)으로 치환한 것이다.[3] 그러나 제주도에서 재배되는 품종인 감귤은 영어로 \'mandarin (orange)\'(맨더린 [오렌지])이라고 일컫는다. 비슷한 예로 중국어 제목인 《苦

In [15]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [16]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

질문: 오늘 날씨 어때?
답변: 맑고 따뜻한 기온이


In [17]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [18]:
# Example
question = "연출은 누가했습니까?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)

질문: 연출은 누가했습니까?
답변: 김원석이


In [19]:
# Example
question = "작품은 어디서 볼 수 있습니까?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)

질문: 작품은 어디서 볼 수 있습니까?
답변: 넷플릭스를
